In [1]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [2]:

"""
some variables needed to be set up by users
"""
conference = 'siggraph'
conference_url = "https://dl.acm.org/toc/tog/2018/37/4" # the conference url to download papers from
chromedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\chromedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\SIGGRAPH-2018-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/SIGGRAPH-2018-ALL


In [3]:
"""
conference urls examples

cvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)
eccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)
eccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)
iccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)
icml: http://proceedings.mlr.press/v119/ (ICML 2020)
neurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)
iclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)
siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)

"""




'\nconference urls examples\n\ncvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)\neccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)\neccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)\niccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)\nicml: http://proceedings.mlr.press/v119/ (ICML 2020)\nneurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)\niclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)\n\n'

In [55]:
driver = webdriver.Chrome(chromedriver_path)  
driver.get(conference_url)

from retrieve_titles_urls_from_websites import *
retrieve = globals()['retrieve_from_'+conference]
print('Retrieving pdf urls. This could take some time...')
pdfurllist, pdfnamelist = retrieve(driver)



Retrieving pdf urls. This could take some time...


In [56]:
# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[-(i+1)])


The first 5 pdf urls:

https://dl.acm.org/doi/pdf/10.1145/3197517.3201359
https://dl.acm.org/doi/pdf/10.1145/3197517.3201331
https://dl.acm.org/doi/pdf/10.1145/3197517.3201290
https://dl.acm.org/doi/pdf/10.1145/3197517.3201337
https://dl.acm.org/doi/pdf/10.1145/3197517.3201396

The last 5 pdf urls:

https://dl.acm.org/doi/pdf/10.1145/3197517.3201326
https://dl.acm.org/doi/pdf/10.1145/3197517.3201399
https://dl.acm.org/doi/pdf/10.1145/3197517.3201302
https://dl.acm.org/doi/pdf/10.1145/3197517.3201350
https://dl.acm.org/doi/pdf/10.1145/3197517.3201283


In [57]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[-(i+1)])


The first 5 pdf titles:

Blended cured quasi-newton for distortion optimization
Progressive parameterizations
Anderson acceleration for geometry optimization and physics simulation
Fast winding numbers for soups and clouds
Voxel cores: efficient, robust, and provably good approximation of 3D medial axes

The last 5 pdf titles:

Toonsynth: example-based synthesis of hand-colored cartoon animations
Online optical marker-based hand tracking with deep labels
Robust solving of optical motion capture data by denoising
Headon: real-time reenactment of human portrait videos
Deep video portraits


In [58]:
print('The number of papers is ', len(pdfnamelist))
assert len(pdfnamelist)==len(pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'

The number of papers is  128


In [8]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
            
            with open(root+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  
            _ = time.sleep(random.uniform(4,5)) # for anti-anti-crawler purpose

Start downloading
0 	 Efficient rendering of layered materials using an atomic decomposition with statistical operators 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201289
1 	 The layer laboratory: a calculus for additive and subtractive composition of anisotropic surface reflectance 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201321
2 	 Rendering specular microgeometry with wave optics 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201351
3 	 Gaussian material synthesis 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201307
4 	 Developability of triangle meshes 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201303
5 	 Shape representation by zippables 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201347
6 	 Gradient-domain volumetric photon density estimation 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201363
7 	 A radiative transfer framework for spatially-correlated materials 	 https://dl.acm.org/doi/pdf/10.1145/3197517.3201282
8 	 Example-based turbulence style transfer 	 https://

KeyboardInterrupt: 